# KeyValueMemory Test Notebook

This notebook demonstrates the functionality of the KeyValueMemory class for the text-to-SQL workflow.

In [1]:
# Import necessary libraries
import sys
import asyncio
import logging
import json
from typing import Dict, Any, List, Optional

# Configure logging
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Import autogen_core components
from autogen_core import CancellationToken
from autogen_core.memory import Memory, MemoryContent, MemoryQueryResult, MemoryMimeType

# Import our KeyValueMemory class
from memory import KeyValueMemory

## Initialize the KeyValueMemory

In [2]:
# Create an instance of KeyValueMemory
memory = KeyValueMemory(name="test_memory")
print(f"KeyValueMemory initialized: {memory}, name: {memory.name}")

2025-05-20 12:13:18,322 - root - DEBUG - [KeyValueMemory] Initialized.


KeyValueMemory initialized: <memory.KeyValueMemory object at 0x7f6898786cf0>, name: test_memory


## Test Basic Operations

Let's test the basic operations of the KeyValueMemory class.

In [3]:

# Test 1: Set and get a string variable
print("\n--- Test 1: Set and get a string variable ---")
await memory.set("greeting", "Hello, world!")
greeting = await memory.get("greeting")
print(f"Set 'greeting' to 'Hello, world!'")
print(f"Retrieved 'greeting': {greeting}")

# Test 2: Set and get a JSON variable
print("\n--- Test 2: Set and get a JSON variable ---")
user_data = {"name": "Alice", "role": "Data Scientist", "skills": ["SQL", "Python", "Machine Learning"]}
await memory.set("user_data", user_data)
retrieved_user_data = await memory.get("user_data")
print(f"Set 'user_data' to: {json.dumps(user_data, indent=2)}")
print(f"Retrieved 'user_data': {json.dumps(retrieved_user_data, indent=2)}")

# Test 3: Update an existing variable
print("\n--- Test 3: Update an existing variable ---")
await memory.set("greeting", "Hello, updated world!")
updated_greeting = await memory.get("greeting")
print(f"Updated 'greeting' to 'Hello, updated world!'")
print(f"Retrieved updated 'greeting': {updated_greeting}")

# Test 4: Get variable with details
print("\n--- Test 4: Get variable with details ---")
greeting_details = await memory.get_with_details("greeting")
print(f"Variable details:")
print(f"  Content: {greeting_details.content}")
print(f"  Mime Type: {greeting_details.mime_type}")
print(f"  Metadata: {greeting_details.metadata}")

# Test 5: Set variable with custom metadata
print("\n--- Test 5: Set variable with custom metadata ---")
await memory.set(
    "sql_query", 
    "SELECT * FROM users WHERE role = 'admin'",
    metadata={"created_by": "test_notebook", "priority": "high"}
)
sql_query_details = await memory.get_with_details("sql_query")
print(f"SQL Query with custom metadata:")
print(f"  Content: {sql_query_details.content}")
print(f"  Metadata: {sql_query_details.metadata}")

# Test 6: Query for content
print("\n--- Test 6: Query using MemoryContent ---")
# Query by metadata
query_content = MemoryContent(
    content="",  # Empty content
    mime_type=MemoryMimeType.TEXT,
    metadata={"created_by": "test_notebook"}
)
result = await memory.query(query_content)
print(f"Query by metadata 'created_by': Found {len(result.results)} results")
for idx, item in enumerate(result.results):
    print(f"  Result {idx+1}: {item.content} (metadata: {item.metadata})")

# Test 7: Clear the memory
print("\n--- Test 7: Clear the memory ---")
await memory.clear()
# Try to retrieve a variable after clearing
cleared_greeting = await memory.get("greeting")
print(f"After clearing, 'greeting' value: {cleared_greeting}")

# Test 8: Add binary data
print("\n--- Test 8: Add binary data ---")
binary_data = b"\x00\x01\x02\x03\x04\x05"
await memory.set("binary_example", binary_data)
retrieved_binary = await memory.get("binary_example")
print(f"Set binary data: {binary_data}")
print(f"Retrieved binary data: {retrieved_binary}")
binary_details = await memory.get_with_details("binary_example")
print(f"Binary mime type: {binary_details.mime_type}")

# Test 9: Test with multiple values in store
print("\n--- Test 9: Multiple values in store ---")
await memory.set("var1", "Value 1")
await memory.set("var2", "Value 2")
await memory.set("var3", "Value 3")
# Now update var1
await memory.set("var1", "Updated Value 1")

# Verify we get the latest value
latest_var1 = await memory.get("var1")
print(f"Latest value of var1: {latest_var1}")

# Test 10: Cancellation token
print("\n--- Test 10: Cancellation token ---")
# Create a cancelled token
token = CancellationToken()
token.cancel()

# Try to add with cancelled token
await memory.set("cancelled_var", "This should not be added")
# Try to add with a cancelled token
content_item = MemoryContent(content="This should not be added", mime_type=MemoryMimeType.TEXT, 
                            metadata={"variable_name": "cancelled_var"})
await memory.add(content_item, token)

# Check if it was added
cancelled_var = await memory.get("cancelled_var")
print(f"Value of cancelled_var: {cancelled_var}")

# Try to query with cancelled token
result = await memory.query("var1", token)
print(f"Query with cancelled token: Found {len(result.results)} results")

2025-05-20 12:13:44,439 - root - DEBUG - [KeyValueMemory] Added content with metadata: {'variable_name': 'greeting'}. Store size: 1
2025-05-20 12:13:44,440 - root - DEBUG - [KeyValueMemory] Set key 'greeting'.
2025-05-20 12:13:44,440 - root - DEBUG - [KeyValueMemory] Querying for variable_name='greeting', found 1 item(s) (returning latest).
2025-05-20 12:13:44,440 - root - DEBUG - [KeyValueMemory] Added content with metadata: {'variable_name': 'user_data'}. Store size: 2
2025-05-20 12:13:44,441 - root - DEBUG - [KeyValueMemory] Set key 'user_data'.
2025-05-20 12:13:44,441 - root - DEBUG - [KeyValueMemory] Querying for variable_name='user_data', found 1 item(s) (returning latest).
2025-05-20 12:13:44,441 - root - DEBUG - [KeyValueMemory] Added content with metadata: {'variable_name': 'greeting'}. Store size: 3
2025-05-20 12:13:44,441 - root - DEBUG - [KeyValueMemory] Set key 'greeting'.
2025-05-20 12:13:44,442 - root - DEBUG - [KeyValueMemory] Querying for variable_name='greeting', foun


--- Test 1: Set and get a string variable ---
Set 'greeting' to 'Hello, world!'
Retrieved 'greeting': Hello, world!

--- Test 2: Set and get a JSON variable ---
Set 'user_data' to: {
  "name": "Alice",
  "role": "Data Scientist",
  "skills": [
    "SQL",
    "Python",
    "Machine Learning"
  ]
}
Retrieved 'user_data': {
  "name": "Alice",
  "role": "Data Scientist",
  "skills": [
    "SQL",
    "Python",
    "Machine Learning"
  ]
}

--- Test 3: Update an existing variable ---
Updated 'greeting' to 'Hello, updated world!'
Retrieved updated 'greeting': Hello, updated world!

--- Test 4: Get variable with details ---
Variable details:
  Content: Hello, updated world!
  Mime Type: MemoryMimeType.TEXT
  Metadata: {'variable_name': 'greeting'}

--- Test 5: Set variable with custom metadata ---
SQL Query with custom metadata:
  Content: SELECT * FROM users WHERE role = 'admin'
  Metadata: {'variable_name': 'sql_query', 'created_by': 'test_notebook', 'priority': 'high'}

--- Test 6: Query u

## Test Advanced Use Cases

Now let's explore some more advanced use cases for the KeyValueMemory class.

In [4]:

# Clear any existing data
await memory.clear()

# Test 1: Simulate storing and retrieving SQL workflow state
print("\n--- Advanced Test 1: SQL workflow state ---")

# Store database connection info
db_config = {
    "db_id": "spider_dev",
    "connection_string": "sqlite:///path/to/database.db",
    "timeout": 30
}
await memory.set("db_config", db_config)

# Store schema information
schema_info = """<database_schema>
<table name="users">
    <column name="id" type="INTEGER" primary_key="true" />
    <column name="username" type="TEXT" />
    <column name="email" type="TEXT" />
</table>
<table name="products">
    <column name="id" type="INTEGER" primary_key="true" />
    <column name="name" type="TEXT" />
    <column name="price" type="REAL" />
</table>
</database_schema>"""
await memory.set("schema_info", schema_info)

# Store user query
await memory.set("user_query", "Show me all users who have purchased products over $100")

# Store generated SQL
sql = """
SELECT users.username, users.email, products.name, products.price
FROM users
JOIN orders ON users.id = orders.user_id
JOIN products ON orders.product_id = products.id
WHERE products.price > 100
ORDER BY products.price DESC
"""
await memory.set("generated_sql", sql)

# Retrieve workflow state
retrieved_query = await memory.get("user_query")
retrieved_sql = await memory.get("generated_sql")
retrieved_schema = await memory.get("schema_info")

print(f"User Query: {retrieved_query}")
print(f"Generated SQL: \n{retrieved_sql}")
print(f"Schema Preview: {retrieved_schema[:60]}...")

# Test 2: Storing execution history with structured metadata
print("\n--- Advanced Test 2: Execution history ---")

# Add execution results with timestamp metadata
for i in range(3):
    execution_result = {
        "status": "success" if i != 1 else "error",
        "records": i * 5,
        "execution_time": 0.5 + (i * 0.2)
    }
    
    if i == 1:
        execution_result["error"] = "Column 'order_date' not found"
        
    await memory.set(
        f"execution_result_{i}", 
        execution_result,
        metadata={
            "timestamp": f"2023-05-20T14:{i}0:00Z",
            "execution_id": f"exec_{i}",
            "status": execution_result["status"]
        }
    )

# Query for error executions
error_query = MemoryContent(
    content="",
    mime_type=MemoryMimeType.JSON,
    metadata={"status": "error"}
)
error_results = await memory.query(error_query)

print(f"Found {len(error_results.results)} error executions:")
for idx, result in enumerate(error_results.results):
    print(f"  Error {idx+1}: {json.dumps(result.content, indent=2)}")
    print(f"  Metadata: {result.metadata}")
    
# Test 3: Simulating variable overrides with history
print("\n--- Advanced Test 3: Variable overrides with history ---")

# Store a sequence of SQL refinements
sql_versions = [
    "SELECT * FROM users",
    "SELECT id, username FROM users WHERE active = true",
    "SELECT id, username, email FROM users WHERE active = true ORDER BY username"
]

for i, sql_version in enumerate(sql_versions):
    await memory.set(
        "current_sql",
        sql_version,
        metadata={
            "version": i + 1,
            "timestamp": f"2023-05-20T15:{i}0:00Z",
            "refinement_reason": ["Initial query", "Added filters", "Improved sorting"][i]
        }
    )

# Get the current SQL (should be the latest version)
current_sql = await memory.get("current_sql")
current_sql_details = await memory.get_with_details("current_sql")

print(f"Current SQL: {current_sql}")
print(f"Version: {current_sql_details.metadata.get('version')}")
print(f"Refinement reason: {current_sql_details.metadata.get('refinement_reason')}")

# Test 4: Store debug information
print("\n--- Advanced Test 4: Debug information ---")

# Let's store some debug logs in memory
debug_logs = [
    "Started schema selection at 2023-05-20T15:00:00Z",
    "Found 5 tables in schema",
    "Selected 3 relevant tables for query",
    "Generated SQL with JOIN between users and orders",
    "Execution completed in 1.2 seconds"
]

for i, log in enumerate(debug_logs):
    await memory.set(
        f"debug_log_{i}",
        log,
        metadata={
            "log_level": "DEBUG",
            "component": ["SchemaSelector", "SchemaSelector", "SchemaSelector", "SQLGenerator", "SQLExecutor"][i]
        }
    )

# Query for logs from the SQLGenerator component
sql_generator_query = MemoryContent(
    content="",
    mime_type=MemoryMimeType.TEXT,
    metadata={"component": "SQLGenerator"}
)
sql_generator_logs = await memory.query(sql_generator_query)

print(f"SQLGenerator logs:")
for log in sql_generator_logs.results:
    print(f"  {log.content}")

2025-05-20 12:14:08,397 - root - INFO - [KeyValueMemory] Memory cleared.
2025-05-20 12:14:08,398 - root - DEBUG - [KeyValueMemory] Added content with metadata: {'variable_name': 'db_config'}. Store size: 1
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Set key 'db_config'.
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Added content with metadata: {'variable_name': 'schema_info'}. Store size: 2
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Set key 'schema_info'.
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Added content with metadata: {'variable_name': 'user_query'}. Store size: 3
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Set key 'user_query'.
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Added content with metadata: {'variable_name': 'generated_sql'}. Store size: 4
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Set key 'generated_sql'.
2025-05-20 12:14:08,399 - root - DEBUG - [KeyValueMemory] Queryin


--- Advanced Test 1: SQL workflow state ---
User Query: Show me all users who have purchased products over $100
Generated SQL: 

SELECT users.username, users.email, products.name, products.price
FROM users
JOIN orders ON users.id = orders.user_id
JOIN products ON orders.product_id = products.id
WHERE products.price > 100
ORDER BY products.price DESC

Schema Preview: <database_schema>
<table name="users">
    <column name="id"...

--- Advanced Test 2: Execution history ---
Found 4 error executions:
  Error 1: "<database_schema>\n<table name=\"users\">\n    <column name=\"id\" type=\"INTEGER\" primary_key=\"true\" />\n    <column name=\"username\" type=\"TEXT\" />\n    <column name=\"email\" type=\"TEXT\" />\n</table>\n<table name=\"products\">\n    <column name=\"id\" type=\"INTEGER\" primary_key=\"true\" />\n    <column name=\"name\" type=\"TEXT\" />\n    <column name=\"price\" type=\"REAL\" />\n</table>\n</database_schema>"
  Metadata: {'variable_name': 'schema_info'}
  Error 2: "Sho